In [1]:
import fitz  # PyMuPDF
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import streamlit as st
import asyncio

# Extract text from PDFs using fitz (PyMuPDF)
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Paths to the PDF files
alphabet_pdf_path = "Alphabet_10K.pdf"
tesla_pdf_path = "Tesla_10K.pdf"
uber_pdf_path = "Uber_10K.pdf"

# Extract text from the PDFs
alphabet_text = extract_text_from_pdf(alphabet_pdf_path)
tesla_text = extract_text_from_pdf(tesla_pdf_path)
uber_text = extract_text_from_pdf(uber_pdf_path)

# Generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
alphabet_embeddings = model.encode(alphabet_text.split('\n'))
tesla_embeddings = model.encode(tesla_text.split('\n'))
uber_embeddings = model.encode(uber_text.split('\n'))

# Store embeddings in Faiss
dimension = 384  # for 'all-MiniLM-L6-v2'
index = faiss.IndexFlatL2(dimension)
all_embeddings = np.vstack([alphabet_embeddings, tesla_embeddings, uber_embeddings])
index.add(all_embeddings)

# Query function
def query_vector_store(query, k=5):
    query_embedding = model.encode([query])[0]
    D, I = index.search(np.array([query_embedding]), k)
    return I

# Local LLM for insights
model_name = 'EleutherAI/gpt-neo-2.7B'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)

async def generate_insights(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=1000)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Streamlit UI
st.title("Content Engine for 10-K Filings")
query = st.text_input("Enter your query:")

# Function to run async function within Streamlit
def run_async_function():
    if query:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        result = loop.run_until_complete(generate_insights(query))
        st.write(result)

# Check if in main thread or running in a script
if __name__ == '__main__':
    run_async_function()

C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


2024-07-01 01:58:29.248 
  command:

    streamlit run C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
